In [0]:
!pip install -q sklearn 

In [0]:
%tensorflow_version 2.x

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output 
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

In [0]:
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv') #training data
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv') #testing data
print(dftrain.head())
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')
print(dftrain.head())

In [0]:
categorical_columns = ['sex', 'n_siblings_spouses', 'parch', 'class','deck','embark_town','alone']
numeric_columns = ['age','fare']

In [14]:
feature_columns = []
for feature_name in categorical_columns:
  vocabulary = dftrain [feature_name].unique()  #gets a list of all unique values from given feature column
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))
  
for feature_name in numeric_columns:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype= tf.float32))

print(feature_columns)

[VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='n_siblings_spouses', vocabulary_list=(1, 0, 3, 4, 2, 5, 8), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='parch', vocabulary_list=(0, 1, 2, 5, 3, 4), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='class', vocabulary_list=('Third', 'First', 'Second'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='deck', vocabulary_list=('unknown', 'C', 'G', 'A', 'B', 'D', 'F', 'E'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Southampton', 'Cherbourg', 'Queenstown', 'unknown'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='alone', vocabulary_list=('n', 'y'), dtype=tf.string, def

In [16]:
def make_input_fn (data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):   #data_df is big dataframe, label_df is y-value(what we are predicting)
  def input_function():     # inner function, this will be returned
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))  #create tf.data.Dataset object with data and its label
    if shuffle:
      ds=ds.shuffle(1000)      #shuffle/randomize order of data
    ds = ds.batch(batch_size).repeat(num_epochs)   #split dataset into batches of 32 and repeat process for number of epochs
    return ds    #return a batch of the dataset
  return input_function      #return a function object for use

train_input_fn = make_input_fn(dftrain, y_train)   #call an input fxn that was returned to us to get a dataset object we can feed to the model // this is the input function
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)

linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)   #pass feature_columns from above into the estimator module to create module

linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)

clear_output()     # clears console output
print(result['accuracy'])   #results variable is simply a dict of stats about our model
print(result)




0.7613636
{'accuracy': 0.7613636, 'accuracy_baseline': 0.625, 'auc': 0.8242118, 'auc_precision_recall': 0.75138813, 'average_loss': 0.5075951, 'label/mean': 0.375, 'loss': 0.50250065, 'precision': 0.6551724, 'prediction/mean': 0.4187341, 'recall': 0.7676768, 'global_step': 200}


In [21]:
result = list(linear_est.predict(eval_input_fn))
print(dfeval.loc[0])   #returns actual characteristics from that index in original dataset
print(y_eval.loc[0])   #returns actual outcome from that index 
print(result[0]['probabilities'][1])   #returns models prediction of survival of that index

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp4d1lolak/model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
sex                          male
age                            35
n_siblings_spouses              0
parch                           0
fare                         8.05
class                       Third
deck                      unknown
emb